In [4]:
using PCT
using Pkg

LoadError: ArgumentError: Package VSCodeServer not found in current path.
- Run `import Pkg; Pkg.add("VSCodeServer")` to install the VSCodeServer package.

In [3]:
using VS
@profview h, _ = @pct pullback((z::C) -> z' * z)

LoadError: LoadError: UndefVarError: `@profview` not defined
in expression starting at In[3]:2

In [4]:
println(verbose(h))

Pullback((z::C->
    (*
        z::C,
        z::C'
    )::C
::[C->C])::[C×C->C]


In [5]:
reduce_pullback(h)[1]

(z, _K) -> 
    ((z, _K) -> 
        (((z, _K) -> 
            _K)(z, (_K⋅z))+((z, _K) -> 
            ((z, _K) -> 
                _K)(z, _K'))(z, (_K⋅z'))))(z, _K)

In [9]:
p = eval_all(reduce_pullback(h)[1])
q = eval_all(call(p, first(ff(p)), constant(1)))

(z⋅2.0)

In [10]:
simplify(q) |> first

  0.000000 seconds


(z⋅2.0)

In [2]:
f, ctx = @pct begin
    @space H begin
        type=(I, I)->C
        symmetries=(((2,1), :conj),)
    end
    
    @space V begin
        type=(I,)->C
    end

    @space T begin
        type=(I,I,I,I) -> C
        symmetries=(((2,1,4,3), :conj), ((3,4,1,2), :id))
    end
                
    @space U begin
        type=(I,I)->C
    end
                    
    (A::H, J::T) -> _
end

(Map(MapType(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))))]), UndeterminedPCTType(), Dict{Any, Any}()), PCTVector(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))))]), Var{MapType}[Var{MapType}(MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict{Symbol, Any}(:name => :H, :symmetries => (((2, 1), :conj),))), :A), Var{MapType}(MapType(VecType(AbstractPCTType[I(), I(), I(), I()]), C(), Dict{Symbol, Any}(:name => :T, :symmetries => (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id)))), :J)]), Var{UndeterminedPCTTyp

In [11]:
f

(A, J) -> 
    _

In [39]:
g = @pct f ctx pullback((x::V) -> sum((i, j), x(i)' * A(i, j) * x(j)))

(A, J) -> 
    𝒫((x) -> 
        ∑((i_0, i_1), (x(i_0)⋅A(i_1, i_0)⋅x(i_1)')))

In [40]:
println(verbose(g))

(A::[I×I->C], J::[I×I×I×I->C]->
    Pullback((x::[I->C]->
        ∑((i_0::I, i_1::I),
            (*
                x::[I->C](i_0::I)::C,
                A::[I×I->C](i_1::I, i_0::I)::C,
                x::[I->C](i_1::I)::C'
            )::C
        )::C
    ::[[I->C]->C])::[[I->C]×C->[I->C]]
::[[I×I->C]×[I×I×I×I->C]->[[I->C]×C->[I->C]]]


In [41]:
p = fc(eval_all(reduce_pullback(g)))

(x, _K) -> 
    (i_2) -> 
        ∑((i_0, i_1), (δ(i_2, i_1, (x(i_0)⋅A(i_1, i_0)⋅_K'))+δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'⋅_K))))

In [42]:
q = eval_all(call(p, first(ff(p)), constant(1)))

(i_2) -> 
    ∑((i_0, i_1), (δ(i_2, i_1, (x(i_0)⋅A(i_1, i_0)))+δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))

In [43]:
simplify(q) |> first

  0.000019 seconds (7 allocations: 304 bytes)
1 sum_dist
∑((i_0, i_1), (δ(i_2, i_1, (x(i_0)⋅A(i_1, i_0)))+δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
(∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)')))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))

  0.000281 seconds (293 allocations: 13.062 KiB)
2 contract_delta
(∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)')))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
directed!

  0.000041 seconds (215 allocations: 9.688 KiB)
3 contract_delta
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0, i_1), δ(i_2, i_0, (x(i_1)⋅A(i_1, i_0)'))))
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0), (x(i_0)⋅A(i_2, i_0))))
directed!

  0.000019 seconds (37 allocations: 1.625 KiB)
4 gcd
(∑((i_0), (x(i_0)⋅A(i_2, i_0)))+∑((i_0), (x(i_0)⋅A(i_2, i_0))))
((1+1)⋅∑((i_0), (x(i_0)⋅A(i_2, i_0))))

  0.000042 seconds (244 allocations: 11.312 KiB)
5 add_const
((1+1)⋅∑((i_0), (x(i_0)⋅A(i_2, i_0))))
(∑((i_0), (x

(i_2) -> 
    (∑((i_0), (x(i_0)⋅A(i_2, i_0)))⋅2)

In [44]:
g = @pct f ctx pullback((C::U) -> sum((i,j,p,q,r,s), C(p,i)' * C(q,i) * C(r,j)' * C(s,j) * J(p,q,r,s)))

(A, J) -> 
    𝒫((C) -> 
        ∑((i_0, i_1, i_2, i_3, i_4, i_5), (C(i_1, i_5)⋅C(i_3, i_4)⋅J(i_1, i_0, i_3, i_2)'⋅C(i_2, i_4)'⋅C(i_0, i_5)')))

In [45]:
p = fc(eval_all(reduce_pullback(g)))

(C, _K) -> 
    (i_6, i_7) -> 
        ∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_7, i_5, δ(i_6, i_2, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_1, i_0)'⋅_K)))+δ(i_7, i_5, δ(i_6, i_3, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_4, i_0)'⋅_K')))+δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_2, i_5)'⋅_K)))+δ(i_7, i_0, δ(i_6, i_4, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_3, i_5)'⋅_K')))))

In [46]:
q = eval_all(call(p, first(ff(p)), constant(1)))

(i_6, i_7) -> 
    ∑((i_0, i_1, i_2, i_3, i_4, i_5), (δ(i_7, i_5, δ(i_6, i_2, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_1, i_0)')))+δ(i_7, i_5, δ(i_6, i_3, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_4, i_0)')))+δ(i_7, i_0, δ(i_6, i_1, (J(i_1, i_4, i_2, i_3)⋅C(i_4, i_0)⋅C(i_3, i_5)⋅C(i_2, i_5)')))+δ(i_7, i_0, δ(i_6, i_4, (C(i_1, i_0)⋅C(i_2, i_5)⋅J(i_1, i_4, i_2, i_3)'⋅C(i_3, i_5)')))))

In [8]:
Profile.clear()
@profile p==p
pprof()

"profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:57599


In [ ]:
(A::H) -> (x::V) -> x A x

In [2]:
f, ctx = @pct begin 

    @domain P begin
        base = I
        lower = -N
        upper = N-1
        periodic = true
    end

    @domain Q begin
        base = I
        lower = -N
        upper = N
        contractable = false
    end

    @space Mmn begin
        type = (I, I, I, I) -> C
        symmetries = (((2, 1, 4, 3), :conj), )
    end

    @space Sym begin
        type = (I, ) -> C
        symmetries = (((1, ), :ineg), )
    end

    @space Gauge begin
        type = (I, I, I) -> C
    end

    @space Density begin
        type = (I, I) -> C
    end

    #= @space SymC begin
        type = (I, ) -> C
        symmetries = (((1, ), :inegc), )
    end =#

    (S::Mmn, w::Sym) -> _
end

g = @pct f ctx pullback((U::Gauge) -> ((ρ::Density) -> sum((n::I, b::Q), ρ(n, b)' * ρ(n, b)))(
    (n::I, b::Q) -> sum((k::P, p, q), U(p, n, k)' * S(p,q,k,k+b) * U(q, n, k+b))))
eval_all(g)
g_1 = fc(eval_all(reduce_pullback(eval_all(g))))
g_2 = eval_all(call(g_1, first(ff(g_1)), constant(1)))

(i_0, i_1, i_2) -> 
    ∑((n, b), (2.0⋅∑((k, p, q), (δ(i_2, (b+k), δ(i_1, n, δ(i_0, q, (S(p, q, k, (b+k))'⋅∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))⋅U(p, n, k)))))+δ(i_2, k, δ(i_1, n, δ(i_0, p, (∑((k, p, q), (U(q, n, (b+k))'⋅S(p, q, k, (b+k))'⋅U(p, n, k)))⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))))))))

In [11]:
g_3 = simplify(g_2) |> first

  6.023939 seconds (71.71 M allocations: 3.285 GiB, 12.83% gc time)
1 sum_out
∑((n, b), (2.0⋅∑((k, p, q), (δ(i_2, (b+k), δ(i_1, n, δ(i_0, q, (S(p, q, k, (b+k))'⋅∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))⋅U(p, n, k)))))+δ(i_2, k, δ(i_1, n, δ(i_0, p, (∑((k, p, q), (U(q, n, (b+k))'⋅S(p, q, k, (b+k))'⋅U(p, n, k)))⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))))))))
∑((b), (2.0⋅∑((n), ∑((k, p, q), (δ(i_2, (b+k), δ(i_1, n, δ(i_0, q, (S(p, q, k, (b+k))'⋅∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))⋅U(p, n, k)))))+δ(i_2, k, δ(i_1, n, δ(i_0, p, (∑((k, p, q), (U(q, n, (b+k))'⋅S(p, q, k, (b+k))'⋅U(p, n, k)))⋅U(q, n, (b+k))⋅S(p, q, k, (b+k)))))))))))
directed!

  8.197918 seconds (97.68 M allocations: 4.473 GiB, 12.91% gc time)
2 sum_out
∑((b), (2.0⋅∑((n), ∑((k, p, q), (δ(i_2, (b+k), δ(i_1, n, δ(i_0, q, (S(p, q, k, (b+k))'⋅∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))⋅U(p, n, k)))))+δ(i_2, k, δ(i_1, n, δ(i_0, p, (∑((k, p, q), (U(q, n, (b+k))'⋅S(p, q, k, (b+k))'

(i_0, i_1, i_2) -> 
    (2.0⋅(∑((b, q), (∑((k, q), (U(q, i_1, (b+k))'⋅∑((p), (S(p, q, k, (b+k))'⋅U(p, i_1, k)))))⋅U(q, i_1, (i_2+b))⋅S(i_0, q, i_2, (i_2+b))))+∑((b, p), (S(p, i_0, (i_2+(-1.0⋅b)), i_2)'⋅∑((k, q), (∑((p), (U(p, i_1, k)'⋅S(p, q, k, (b+k))))⋅U(q, i_1, (b+k))))⋅U(p, i_1, (i_2+(-1.0⋅b)))))))

In [34]:
using Profile
using Pkg
Pkg.add("PProf")
using PProf

   Resolving package versions...
   Installed XML2_jll ────────────────── v2.12.0+0
   Installed Glib_jll ────────────────── v2.76.5+0
   Installed Xorg_xtrans_jll ─────────── v1.5.0+0
   Installed Libiconv_jll ────────────── v1.17.0+0
   Installed Pango_jll ───────────────── v1.50.14+0
   Installed ProgressMeter ───────────── v1.9.0
   Installed Xorg_libpthread_stubs_jll ─ v0.1.1+0
   Installed ProtoBuf ────────────────── v1.0.14
   Installed Xorg_libX11_jll ─────────── v1.8.6+0
   Installed BufferedStreams ─────────── v1.2.1
   Installed PProf ───────────────────── v3.1.0
   Installed FreeType2_jll ───────────── v2.13.1+0
   Installed Xorg_libXau_jll ─────────── v1.0.11+0
   Installed Xorg_libXdmcp_jll ───────── v1.1.4+0
   Installed Xorg_libxcb_jll ─────────── v1.15.0+0
   Installed CodecZlib ───────────────── v0.7.3
    Updating `~/kaer_morhen/projects/QLD/PCT/Project.toml`
  [e4faabce] + PProf v3.1.0
    Updating `~/kaer_morhen/projects/QLD/PCT/Manifest.toml`
  [1520ce14] + Abstra

┌ Warning: Module JLLWrappers with build ID fafbfcfd-cb8c-49c8-0005-da57d32b1727 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1793
[ Info: Skipping precompilation since __precompile__(false). Importing pprof_jll [cf2c5f97-e748-59fa-a03f-dda3c62118cb].
[ Info: Precompiling Graphviz_jll [3c863552-8265-54e4-a6dc-903eb78fde85]
┌ Warning: Module JLLWrappers with build ID fafbfcfd-cb8c-49c8-0005-da57d32b1727 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1793
[ Info: Skipping precompilation since __precompile__(false). Importing Graphviz_jll [3c863552-8265-54e4-a6dc-903eb78fde85].
[ Info: Precompiling Cairo_jll [83423d85-b0ee-5818-9007-b63ccbeb887a]
┌ Warning: Module JLLWrappers with build ID fafbfcfd-cb8c-49c8-0005-da

┌ Warning: Module JLLWrappers with build ID fafbfcfd-cb8c-49c8-0005-da57d32b1727 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1793
[ Info: Skipping precompilation since __precompile__(false). Importing Libuuid_jll [38a345b3-de98-5d2b-a5d3-14cd9215e700].
[ Info: Precompiling Expat_jll [2e619515-83b5-522b-bb60-26c02a35a201]
┌ Warning: Module JLLWrappers with build ID fafbfcfd-cb8c-49c8-0005-da57d32b1727 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1793
[ Info: Skipping precompilation since __precompile__(false). Importing Expat_jll [2e619515-83b5-522b-bb60-26c02a35a201].
[ Info: Precompiling LZO_jll [dd4b983a-f0e5-5f8d-a1b7-129d4a5fb1ac]
┌ Warning: Module JLLWrappers with build ID fafbfcfd-cb8c-49c8-0005-da57d32b

In [88]:
@profile g_4 = simplify(g_3; settings=Dict(:symmetry=>true)) |> first

  0.970068 seconds (9.77 M allocations: 460.792 MiB, 4.28% gc time)
1 sum_sym
((∑((b, q), (U(q, i_1, (b+i_2))⋅S(i_0, q, i_2, (b+i_2))⋅∑((k, q), (U(q, i_1, (b+k))'⋅∑((p), (U(p, i_1, k)⋅S(p, q, k, (b+k))'))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
((∑((b, q), (U(q, i_1, (i_2+(b⋅-1.0)))⋅S(i_0, q, i_2, (i_2+(b⋅-1.0)))⋅∑((k, q), (U(q, i_1, (k+(b⋅-1.0)))'⋅∑((p), (U(p, i_1, k)⋅S(p, q, k, (k+(b⋅-1.0)))'))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
undirected

  1.668342 seconds (17.94 M allocations: 844.179 MiB, 4.78% gc time)
2 symmetry
((∑((b, q), (U(q, i_1, (i_2+(b⋅-1.0)))⋅S(i_0, q, i_2, (i_2+(b⋅-1.0)))⋅∑((k, q), (U(q, i_1, (k+(b⋅-1.0)))'⋅∑((p), (U(p, i_1, k)⋅S(p, q, k, (k+(b⋅-1.0)))'))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q

  1.492732 seconds (17.10 M allocations: 802.470 MiB, 5.35% gc time)
13 shift
((∑((k), (∑((i_0), U(i_0, i_1, (k+(i_4⋅-1.0)))')⋅∑((q, i_5), (U(q, i_1, (i_2+(i_5⋅-1.0)))⋅S(q, i_0, (i_2+(i_5⋅-1.0)), i_2)'⋅∑((p), (U(p, i_1, (k+i_5+(i_4⋅-1.0)))⋅S(p, q, (k+i_5+(i_4⋅-1.0)), (k+(i_4⋅-1.0)))'))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
((∑((k), (∑((i_0), U(i_0, i_1, (k+(i_5⋅-1.0)))')⋅∑((q, i_3), (U(q, i_1, (i_2+(i_3⋅-1.0)))⋅S(q, i_0, (i_2+(i_3⋅-1.0)), i_2)'⋅∑((p), (U(p, i_1, (i_3+k+(i_5⋅-1.0)))⋅S(p, q, (i_3+k+(i_5⋅-1.0)), (k+(i_5⋅-1.0)))'))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
undirected

  1.329935 seconds (15.19 M allocations: 713.014 MiB, 4.53% gc time)
14 shift
((∑((k), (∑((i_0), U(i_0, i_1, (k+(i_5⋅-1.0)))')⋅∑((q, i_3), (U(q, i_1, (i_2+(i_3⋅-1.0)))⋅S(q, i_0, (i_2+(i_

  2.604033 seconds (30.69 M allocations: 1.404 GiB, 4.65% gc time, 0.00% compilation time)
24 shift
((∑((k), (∑((i_0), U(i_0, i_1, (k+(i_3⋅-1.0)))')⋅∑((i_4, p), (U(p, i_1, (i_4+k+(i_3⋅-1.0)))⋅∑((q), (U(q, i_1, (i_2+(i_4⋅-1.0)))⋅S(q, i_0, (i_2+(i_4⋅-1.0)), i_2)'⋅S(p, q, (i_4+k+(i_3⋅-1.0)), (k+(i_3⋅-1.0)))'))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
((∑((k), (∑((i_0), U(i_0, i_1, (k+(i_4⋅-1.0)))')⋅∑((i_5, p), (U(p, i_1, (k+i_5+(i_4⋅-1.0)))⋅∑((q), (U(q, i_1, (i_2+(i_5⋅-1.0)))⋅S(q, i_0, (i_2+(i_5⋅-1.0)), i_2)'⋅S(p, q, (k+i_5+(i_4⋅-1.0)), (k+(i_4⋅-1.0)))'))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
undirected

  2.680874 seconds (31.76 M allocations: 1.453 GiB, 4.36% gc time)
25 shift
((∑((k), (∑((i_0), U(i_0, i_1, (k+(i_4⋅-1.0)))')⋅∑((i_5, p), (U(p, i_1, (k+i_5+(i_4⋅-1.0

  2.811451 seconds (32.25 M allocations: 1.476 GiB, 4.82% gc time)
35 shift
((∑((k), (∑((i_0), U(i_0, i_1, k)')⋅∑((i_3, q), (U(q, i_1, (i_2+(i_3⋅-1.0)))⋅S(i_0, q, i_2, (i_2+(i_3⋅-1.0)))⋅∑((p), (U(p, i_1, (i_3+k))⋅S(p, q, (i_3+k), k)'))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
((∑((k), (∑((i_0), U(i_0, i_1, (k+(i_3⋅-1.0)))')⋅∑((i_4, q), (U(q, i_1, (i_2+(i_4⋅-1.0)))⋅S(i_0, q, i_2, (i_2+(i_4⋅-1.0)))⋅∑((p), (U(p, i_1, (i_4+k+(i_3⋅-1.0)))⋅S(p, q, (i_4+k+(i_3⋅-1.0)), (k+(i_3⋅-1.0)))'))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
undirected

  3.700201 seconds (43.32 M allocations: 1.982 GiB, 4.18% gc time)
36 shift
((∑((k), (∑((i_0), U(i_0, i_1, (k+(i_3⋅-1.0)))')⋅∑((i_4, q), (U(q, i_1, (i_2+(i_4⋅-1.0)))⋅S(i_0, q, i_2, (i_2+(i_4⋅-1.0)))⋅∑((p), (U(p, i_1, (i_4+k+(i_3⋅-1.0)))⋅S

  3.227031 seconds (37.71 M allocations: 1.729 GiB, 4.27% gc time)
46 shift
((∑((k), (∑((i_0), U(i_0, i_1, (i_4+k))')⋅∑((i_3, q), (U(q, i_1, (i_3+i_2))⋅S(i_0, q, i_2, (i_3+i_2))⋅∑((p), (U(p, i_1, (i_4+k+(i_3⋅-1.0)))⋅S(p, q, (i_4+k+(i_3⋅-1.0)), (i_4+k))'))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
((∑((k), (∑((i_0), U(i_0, i_1, (i_3+k))')⋅∑((i_5, q), (U(q, i_1, (i_5+i_2))⋅S(i_0, q, i_2, (i_5+i_2))⋅∑((p), (U(p, i_1, (i_3+k+(i_5⋅-1.0)))⋅S(p, q, (i_3+k+(i_5⋅-1.0)), (i_3+k))'))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
undirected

  3.250391 seconds (38.06 M allocations: 1.745 GiB, 4.83% gc time)
47 shift
((∑((k), (∑((i_0), U(i_0, i_1, (i_3+k))')⋅∑((i_5, q), (U(q, i_1, (i_5+i_2))⋅S(i_0, q, i_2, (i_5+i_2))⋅∑((p), (U(p, i_1, (i_3+k+(i_5⋅-1.0)))⋅S(p, q, (i_3+k+(i_5⋅-1.0)), (i

...

  5.705979 seconds (65.90 M allocations: 3.013 GiB, 4.63% gc time)
102 shift
((∑((k), (∑((i_0), U(i_0, i_1, (k+(i_5⋅-1.0)))')⋅∑((i_3, p), (U(p, i_1, (i_3+k+(i_5⋅-1.0)))⋅∑((q), (U(q, i_1, (i_2+(i_3⋅-1.0)))⋅S(q, p, (k+(i_5⋅-1.0)), (i_3+k+(i_5⋅-1.0)))⋅S(i_0, q, i_2, (i_2+(i_3⋅-1.0)))))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
((∑((k), (∑((i_0), U(i_0, i_1, k)')⋅∑((i_3, p), (U(p, i_1, (i_3+k))⋅∑((q), (U(q, i_1, (i_2+(i_3⋅-1.0)))⋅S(q, p, k, (i_3+k))⋅S(i_0, q, i_2, (i_2+(i_3⋅-1.0)))))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
undirected

  5.025871 seconds (56.63 M allocations: 2.591 GiB, 4.41% gc time)
103 shift
((∑((k), (∑((i_0), U(i_0, i_1, k)')⋅∑((i_3, p), (U(p, i_1, (i_3+k))⋅∑((q), (U(q, i_1, (i_2+(i_3⋅-1.0)))⋅S(q, p, k, (i_3+k))⋅S(i_0, q, i_2, (i_2+(i_3⋅-1.0)))))))))

  4.389569 seconds (48.90 M allocations: 2.238 GiB, 3.96% gc time)
113 shift
((∑((k), (∑((i_0), U(i_0, i_1, k)')⋅∑((i_4, p), (U(p, i_1, (k+(i_4⋅-1.0)))⋅∑((q), (U(q, i_1, (i_4+i_2))⋅S(q, p, k, (k+(i_4⋅-1.0)))⋅S(i_0, q, i_2, (i_4+i_2))))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
((∑((k), (∑((i_0), U(i_0, i_1, (i_4+k))')⋅∑((i_3, p), (U(p, i_1, (i_4+k+(i_3⋅-1.0)))⋅∑((q), (U(q, i_1, (i_3+i_2))⋅S(q, p, (i_4+k), (i_4+k+(i_3⋅-1.0)))⋅S(i_0, q, i_2, (i_3+i_2))))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
undirected

  4.577851 seconds (53.54 M allocations: 2.453 GiB, 4.18% gc time)
114 shift
((∑((k), (∑((i_0), U(i_0, i_1, (i_4+k))')⋅∑((i_3, p), (U(p, i_1, (i_4+k+(i_3⋅-1.0)))⋅∑((q), (U(q, i_1, (i_3+i_2))⋅S(q, p, (i_4+k), (i_4+k+(i_3⋅-1.0)))⋅S(i_0, q, i_2, (i_3+i_2))))))))+∑((b, p

  4.639896 seconds (52.77 M allocations: 2.416 GiB, 4.02% gc time)
124 shift
((∑((k), (∑((i_0), U(i_0, i_1, k)')⋅∑((i_5, q), (U(q, i_1, (i_2+(i_5⋅-1.0)))⋅S(i_0, q, i_2, (i_2+(i_5⋅-1.0)))⋅∑((p), (U(p, i_1, (k+i_5))⋅S(q, p, k, (k+i_5))))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
((∑((k), (∑((i_0), U(i_0, i_1, (k+(i_5⋅-1.0)))')⋅∑((i_3, q), (U(q, i_1, (i_2+(i_3⋅-1.0)))⋅S(i_0, q, i_2, (i_2+(i_3⋅-1.0)))⋅∑((p), (U(p, i_1, (i_3+k+(i_5⋅-1.0)))⋅S(q, p, (k+(i_5⋅-1.0)), (i_3+k+(i_5⋅-1.0)))))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
undirected

  5.295202 seconds (62.39 M allocations: 2.854 GiB, 4.23% gc time)
125 sum_sym
((∑((k), (∑((i_0), U(i_0, i_1, (k+(i_5⋅-1.0)))')⋅∑((i_3, q), (U(q, i_1, (i_2+(i_3⋅-1.0)))⋅S(i_0, q, i_2, (i_2+(i_3⋅-1.0)))⋅∑((p), (U(p, i_1, (i_3+k+(i_5⋅-1.0)))

  5.663918 seconds (67.01 M allocations: 3.068 GiB, 4.31% gc time)
135 sum_sym
((∑((k), (∑((i_0), U(i_0, i_1, (k+i_5))')⋅∑((i_3, p), (U(p, i_1, (i_3+k+i_5))⋅∑((q), (U(q, i_1, (i_2+(i_3⋅-1.0)))⋅S(q, p, (k+i_5), (i_3+k+i_5))⋅S(i_0, q, i_2, (i_2+(i_3⋅-1.0)))))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)
((∑((k), (∑((i_0), U(i_0, i_1, (k+i_5))')⋅∑((i_3, p), (U(p, i_1, (i_3+k+i_5))⋅∑((q), (U(q, i_1, (i_2+(i_3⋅-1.0)))⋅S(q, p, (k+i_5), (i_3+k+i_5))⋅S(i_0, q, i_2, (i_2+(i_3⋅-1.0)))))))))+∑((b, p), (U(p, i_1, (b+i_2))⋅S(p, i_0, (b+i_2), i_2)'⋅∑((k, q), (U(q, i_1, (k+(b⋅-1.0)))⋅∑((p), (S(p, q, k, (k+(b⋅-1.0)))⋅U(p, i_1, k)')))))))⋅2.0)
undirected



LoadError: MethodError: [0mCannot `convert` an object of type [92mBool[39m[0m to an object of type [91mAbstractPCTType[39m

[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mBase.jl:64[24m[39m


In [36]:
pprof()

┌ Warning: The profile data buffer is full; profiling probably terminated
│ before your program finished. To profile for longer runs, call
│ `Profile.init()` with a larger buffer and/or larger delay.
└ @ Profile /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/Profile/src/Profile.jl:639


"profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:57599


In [13]:
settings = Dict(:symmetry => true)
nb(n) = neighbors(n; settings=settings)
fc(g_3)

(2.0⋅(∑((b, q), (∑((k, q), (U(q, i_1, (b+k))'⋅∑((p), (S(p, q, k, (b+k))'⋅U(p, i_1, k)))))⋅U(q, i_1, (i_2+b))⋅S(i_0, q, i_2, (i_2+b))))+∑((b, p), (S(p, i_0, (i_2+(-1.0⋅b)), i_2)'⋅∑((k, q), (∑((p), (U(p, i_1, k)'⋅S(p, q, k, (b+k))))⋅U(q, i_1, (b+k))))⋅U(p, i_1, (i_2+(-1.0⋅b)))))))

In [19]:
@profview fc(g_3) |> nb

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile  -  11:21:24", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip

In [70]:
y = ((nb(fc(g_3))[2] |> first |> nb)[1] |> first |> nb)[4] |> first

((∑((b, q), (U(q, i_1, (i_2+(b⋅-1.0)))⋅S(q, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (k+(b⋅-1.0)))'⋅∑((p), (U(p, i_1, k)⋅S(q, p, (k+(b⋅-1.0)), k)))))))+∑((b, p), (U(p, i_1, (i_2+(b⋅-1.0)))⋅S(p, i_0, (i_2+(b⋅-1.0)), i_2)'⋅∑((k, q), (U(q, i_1, (b+k))⋅∑((p), (S(p, q, k, (b+k))⋅U(p, i_1, k)')))))))⋅2.0)

In [92]:
@profile (y |> nb)
pprof()

┌ Warning: The profile data buffer is full; profiling probably terminated
│ before your program finished. To profile for longer runs, call
│ `Profile.init()` with a larger buffer and/or larger delay.
└ @ Profile /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/Profile/src/Profile.jl:639


"profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:57599
no matches found for regexp: 
